In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from matplotlib import pyplot as plt
import pickle
import sys
sys.path.append('./../')
sys.path.append('./../../')
import Portfolio_value as pv
# import import_ipynb

In [2]:
df =  pd.read_csv('./data/data.csv')
folder_name = 'dyna'


In [3]:
p_val = np.load('./result/{}/p_val/p_val_0.08.npy'.format(folder_name), allow_pickle=True)
w_val = np.load('./result/{}/w_val/w_val_0.08.npy'.format(folder_name), allow_pickle=True)

p_val = p_val.reshape(1)
p_val = p_val[0]

w_val = w_val.reshape(1)
w_val = w_val[0]

In [4]:
bear_year = np.load('./result/year/bear_year.npy', allow_pickle=True).reshape(1)[0]


In [5]:
upper_years = [35]

In [6]:
model_names = ['CPPI_100_5', 'CPPI_100_10', 'CPPI_100_15','CPPI_100_20','CPPI_100_25','CPPI_100_30', 'CPPI_100_35',
 'CPPI_100_40', 'CPPI_100_45', 'CPPI_100_50']
for protection_level in [1.0]:
    for multiplier in range(5,55,5):
        model_names.append('CPPI_{}_{}'.format(int(protection_level*100), multiplier))

In [7]:
bear_perform = {}
for model_name in model_names:
    bear_perform[model_name] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}

In [8]:
# PI_Korea랑은 코드가 좀 다름

def get_metric_for_PI(retire_year, k, upper_year, model_name, withrate, const_or_dyna, df, init_wealth, p_val, w_val):
    # bequest
    
    p_amounts = p_val['{}_rate{}_{}'.format(model_name, withrate, const_or_dyna)][retire_year]
    bequest = p_amounts[12*upper_year]*init_wealth*(1-withrate)/100
    
    # withdraw_amount
    w_amounts = w_val['{}_rate{}_{}'.format(model_name, withrate, const_or_dyna)][retire_year]
    withdraw_amount = sum(w_amounts)
    
    # depletion time
    if np.argwhere(p_amounts == 0).size > 0 : # 고갈이 되면
        dep_time = np.argwhere(p_amounts == 0)[0][0]
    elif np.argwhere(p_amounts == 0).size == 0: # 고갈이 안되면
        dep_time = upper_year*12
        
    # max decline
    dec = np.min(p_amounts)/p_amounts[0]
    dec = dec - 1
    
    # underwater duration
    start_date = datetime(retire_year, 1, 1)
    end_date = start_date + relativedelta(years=upper_year)
    
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m')
    df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    
    under_count = 0
    for i in range(upper_year*12):
        if p_amounts[i]*df.iloc[i]['CPI'] < p_amounts[0]*df.iloc[0]['CPI'] :
             under_count+= 1
        
    return bequest / init_wealth * 100, withdraw_amount / init_wealth * 100, dep_time, dec * 100, under_count

In [9]:
init_wealth = 500000
withdraw_rate = 0.08
k=0
const_or_dyna = 'dyna'
upper_year = 35

for model_name in model_names:
    print(model_name)
    for years in bear_year[upper_year]:
        for year in years:
            res = get_metric_for_PI(year, k, upper_year, model_name, withdraw_rate, const_or_dyna, df, init_wealth, p_val, w_val)
            if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                bear_perform[model_name]['dep'].append(100)
            elif res[2] >= upper_year*12: # 고갈이 안되면
                bear_perform[model_name]['dep'].append(0)
            bear_perform[model_name]['max'].append(res[3])
            bear_perform[model_name]['under'].append(res[4])
            bear_perform[model_name]['inc'].append(res[1])
            bear_perform[model_name]['beq'].append(res[0])
            bear_perform[model_name]['total'].append(res[1] + res[0])


CPPI_100_5
CPPI_100_10
CPPI_100_15
CPPI_100_20
CPPI_100_25
CPPI_100_30
CPPI_100_35
CPPI_100_40
CPPI_100_45
CPPI_100_50


In [10]:
# Define all dictionaries and indices we want to calculate mean for
dicts = [bear_perform]
indices = model_names

# Use list comprehension to compute all means at once
# Store results in lists
a = [np.round(np.mean(d[i]['dep']),1) for i in indices for d in dicts]
b = [np.round(np.mean(d[i]['max']),1) for i in indices for d in dicts]
c = [np.round(np.mean(d[i]['under']),1) for i in indices for d in dicts]
d = [np.round(np.mean(d[i]['inc']),1) for i in indices for d in dicts]
e = [np.round(np.mean(d[i]['beq']),1) for i in indices for d in dicts]
f = [np.round(np.mean(d[i]['total']),1) for i in indices for d in dicts]

# If you need to access the results separately, you can do so by indexing the lists:
# For example, a1 corresponds to a_results[0], a2 corresponds to a_results[1], etc.

In [11]:

print(model_names)
print('Depletion Prob. (\%) & & ' + ' & '.join(['{}'] * len(a)).format(*a))
print('Max. decline (\%) & & ' + ' & '.join(['{}'] * len(b)).format(*b))
print('Underwater duration (month) & & ' + ' & '.join(['{}'] * len(c)).format(*c))
print('Total withdrawal (\%) & & ' + ' & '.join(['{}'] * len(d)).format(*d))
print('Bequest (\%) & & ' + ' & '.join(['{}'] * len(e)).format(*e))
print('Total withdrawal + Bequest (\%) & & ' + ' & '.join(['{}'] * len(f)).format(*f))





['CPPI_100_5', 'CPPI_100_10', 'CPPI_100_15', 'CPPI_100_20', 'CPPI_100_25', 'CPPI_100_30', 'CPPI_100_35', 'CPPI_100_40', 'CPPI_100_45', 'CPPI_100_50']
Depletion Prob. (\%) & & 100.0 & 100.0 & 90.9 & 77.3 & 68.2 & 68.2 & 63.6 & 59.1 & 59.1 & 59.1
Max. decline (\%) & & -100.0 & -100.0 & -98.5 & -95.3 & -90.0 & -88.4 & -86.4 & -84.7 & -83.3 & -82.5
Underwater duration (month) & & 398.1 & 383.2 & 356.4 & 330.4 & 279.1 & 269.5 & 268.4 & 264.2 & 259.6 & 251.0
Total withdrawal (\%) & & 106.1 & 112.6 & 116.8 & 121.2 & 125.6 & 127.3 & 130.3 & 131.6 & 132.8 & 133.6
Bequest (\%) & & 0.0 & 0.0 & 1.5 & 4.7 & 10.9 & 14.6 & 18.6 & 21.5 & 24.0 & 26.2
Total withdrawal + Bequest (\%) & & 106.1 & 112.6 & 118.3 & 126.0 & 136.5 & 141.9 & 149.0 & 153.1 & 156.8 & 159.8
